In [1]:

%pip install --upgrade pip
%pip install python-dotenv jq gdown
%pip install langchain langchain_openai langchain_community
%pip install scikit-learn pandas numpy dask datasets docarray
%pip install bertopic[all]
%pip install --upgrade ipywidgets notebook
%pip install "accelerate>=0.26.0"
!jupyter nbextension enable --py widgetsnbextension --sys-prefix







Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [2]:
import os
import pandas as pd
import numpy as np
import dask.dataframe as dd
import gdown
import sklearn
import langchain
import datasets
import docarray
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

print("All packages imported successfully!")



All packages imported successfully!


In [3]:
# Mount Google Drive (For Colab Users)
#from google.colab import drive
#drive.mount('/content/drive')


In [4]:
amazonhkdatasetfileid = '14GcJAzyN2PFg2JuyzF0pRmxlMmimrz9o'
amazonhkdatasetfilename = 'AmazonHomeKitchenReviews.csv'

url = f"https://drive.google.com/uc?export=download&id={amazonhkdatasetfileid}"

gdown.download(url,amazonhkdatasetfilename, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?export=download&id=14GcJAzyN2PFg2JuyzF0pRmxlMmimrz9o
From (redirected): https://drive.google.com/uc?export=download&id=14GcJAzyN2PFg2JuyzF0pRmxlMmimrz9o&confirm=t&uuid=a5f12988-8b5c-4415-8b08-4133854d0c0d
To: c:\Users\spscd\OneDrive\Desktop\AmazonHomeKitchenReviews.csv
100%|██████████| 692M/692M [00:13<00:00, 52.9MB/s] 


'AmazonHomeKitchenReviews.csv'

* Read the dataset csv  into dataframes

In [5]:
df_data = pd.read_csv(amazonhkdatasetfilename)


In [6]:
print(len(df_data))

754079


In [7]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754079 entries, 0 to 754078
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         754079 non-null  int64  
 1   rating             754079 non-null  int64  
 2   title_x            753916 non-null  object 
 3   text               753888 non-null  object 
 4   images             754079 non-null  object 
 5   asin               754079 non-null  object 
 6   parent_asin        754079 non-null  object 
 7   user_id            754079 non-null  object 
 8   timestamp          754079 non-null  object 
 9   helpful_vote       754079 non-null  int64  
 10  verified_purchase  754079 non-null  bool   
 11  title_y            754079 non-null  object 
 12  description        754079 non-null  object 
 13  price              754079 non-null  float64
 14  Brand              754077 non-null  object 
 15  Material           754079 non-null  object 
 16  Co

In [8]:
#reduce the dataset to 10000 records
#df_data = df_data.head(10000)



*   Analyse the datasets
*  Check total number of unique products and the review counts




In [9]:
df_renamed = df_data.rename(columns={'title_y' : 'product_title','title_x':'review_title','text':'review_text'})
df_renamed.groupby('product_title').size().sort_values(ascending=False).head(5)
print(df_renamed.columns)

Index(['Unnamed: 0', 'rating', 'review_title', 'review_text', 'images', 'asin',
       'parent_asin', 'user_id', 'timestamp', 'helpful_vote',
       'verified_purchase', 'product_title', 'description', 'price', 'Brand',
       'Material', 'Color', 'categories'],
      dtype='object')


* Pre-processing
* X = review_title,review_text
* y = rating

In [10]:
print(df_renamed.value_counts('price'))




price
19.99      25494
14.99      23353
29.99      21455
24.99      21141
34.97      20431
           ...  
651.68         1
2665.48        1
1015.16        1
799.00         1
795.00         1
Name: count, Length: 3013, dtype: int64


In [11]:
qa_pairs = []
for index , row in df_renamed.iterrows():
  context = str(row['review_title']) + ". " + str(row["review_text"]) + " " + "$" + str(row['price']) + ". " + str(row['rating']) +" " +  str(row['product_title'])
  #print(context

    # Generate questions about user rating only if rating is not null
  if not pd.isna(row['rating']):
        qa_pairs.append({
            'context': context,
            'question': f"What was the average rating for product{str(row['product_title'])}?",
          #  'answer' : str(row['rating'])
            'answer': {'text': str(row['rating']), 'answer_start': context.find(str(row['rating'])) if str(row['rating']) in context else 0}

        })

        # Question about why they gave that rating
        qa_pairs.append({
            'context': context,
            'question': f"Why did the customer rate this product {str(row['rating'])} stars?",
             'answer': {'text': context[:150], 'answer_start': 0}
        })



In [12]:
print(qa_pairs[5:12])

[{'context': 'Pretty colors available. Nice thin placemats of good size. Can be used also as table doilies. $16.99. 5 SHACOS Round Braided Placemats Set of 6 Round Table Mats for Dining Tables 15 inch Washable Non Slip Placemats (Pink, 6)', 'question': 'Why did the customer rate this product 5 stars?', 'answer': {'text': 'Pretty colors available. Nice thin placemats of good size. Can be used also as table doilies. $16.99. 5 SHACOS Round Braided Placemats Set of 6 Round ', 'answer_start': 0}}, {'context': 'Nice material. Very pretty, wish they came bigger $6.92. 4 PaperLanternStore.com 7 Inch Bloom Shaped Handmade Cotton Crochet Doilies - Beige (2 PACK)', 'question': 'What was the average rating for productPaperLanternStore.com 7 Inch Bloom Shaped Handmade Cotton Crochet Doilies - Beige (2 PACK)?', 'answer': {'text': '4', 'answer_start': 57}}, {'context': 'Nice material. Very pretty, wish they came bigger $6.92. 4 PaperLanternStore.com 7 Inch Bloom Shaped Handmade Cotton Crochet Doilies

In [13]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model='distilbert-base-uncased-distilled-squad')

Device set to use cpu


In [14]:
result = question_answerer(question="What is the average rating for dinner mugs", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: '$31.99. 3', score: 0.077, start: 211, end: 220


In [15]:
result = question_answerer(question="What are customer saying about dinner mug", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'I wouldn't buy them for my home or for frequent use', score: 0.1069, start: 158, end: 209




* Load the QA dataset
* Split the dataset for train test




In [16]:
from datasets import Dataset

dataset = Dataset.from_list(qa_pairs)
dataset = dataset.train_test_split(test_size=0.2)

print(dataset.keys())
print(dataset['train'][0])


dict_keys(['train', 'test'])
{'context': 'AMAZING ADDITION. Great size, really extra big inside and yet not taking up anymore standing room than a regular can.  Love the way it opens immediately when you approach it which is very helpful when your hands are messy from cooking or too full to open it.  Also very attractive in the kitchen.  Itdoes not leave finger prints if you do touch it, another plus. $80.44. 5 iTouchless 13 Gallon Pet-Proof Sensor Trash Can with AbsorbX Odor Filter Kitchen Garbage Bin Prevents Dogs & Cats Opening Lid, Plus, Stainless Steel with PetGuard', 'question': 'What was the average rating for productiTouchless 13 Gallon Pet-Proof Sensor Trash Can with AbsorbX Odor Filter Kitchen Garbage Bin Prevents Dogs & Cats Opening Lid, Plus, Stainless Steel with PetGuard?', 'answer': {'answer_start': 371, 'text': '5'}}


* Tokenize the dataset

In [17]:
from transformers import DistilBertTokenizerFast

# Load the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-cased-distilled-squad")

# Function to tokenize and compute positions
def tokenize_and_align(batch):
    tokenized_inputs = tokenizer(
        batch["question"],
        batch["context"],
        truncation=True,
        max_length=512,
        return_offsets_mapping=True,  # To map tokens to character positions
        padding="max_length"  # Optional: Ensures all inputs are the same length
    )

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(tokenized_inputs["offset_mapping"]):
        answer = batch["answer"][i]
        answer_start = answer["answer_start"]
        answer_text = answer["text"]

        # Find the start and end token indices
        start_token_idx, end_token_idx = None, None
        for idx, (start, end) in enumerate(offsets):
            if start <= answer_start < end:
                start_token_idx = idx
            if start < answer_start + len(answer_text) <= end:
                end_token_idx = idx
                break

        # Default to 0 if answer is not found (e.g., truncated context)
        if start_token_idx is None or end_token_idx is None:
            start_token_idx, end_token_idx = 0, 0

        start_positions.append(start_token_idx)
        end_positions.append(end_token_idx)

    # Add the positions to the tokenized inputs
    tokenized_inputs["start_positions"] = start_positions
    tokenized_inputs["end_positions"] = end_positions
    tokenized_inputs.pop("offset_mapping")  # Remove offsets if not needed

    return tokenized_inputs


In [18]:
# Apply the tokenization function
tokenized_dataset = dataset.map(tokenize_and_align, batched=True, remove_columns=['context', 'question', 'answer'])


Map:   0%|          | 0/1206526 [00:00<?, ? examples/s]

Map:   0%|          | 0/301632 [00:00<?, ? examples/s]

In [19]:
print(tokenized_dataset)
print(tokenized_dataset["train"].column_names)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1206526
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 301632
    })
})
['input_ids', 'attention_mask', 'start_positions', 'end_positions']


* Load the pretrained model

In [20]:
# Save the tokenized dataset to disk in Arrow format
tokenized_dataset.save_to_disk("saved_tokenized_dataset")


Saving the dataset (0/7 shards):   0%|          | 0/1206526 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/301632 [00:00<?, ? examples/s]

In [21]:
model_checkpoint = "distilbert-base-uncased-distilled-squad"

In [22]:
from transformers import DistilBertForQuestionAnswering

model = DistilBertForQuestionAnswering.from_pretrained(model_checkpoint)



*   Define Training Arguments




In [23]:

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./amazonhkqa_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False

)


* Define the trainer and train the model with our dataset

In [24]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer
)


In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 